In [3]:
import numpy as np
f64 = np.float64

## Metropolis-Hastings
I would like to try and generate the following distribution.
\begin{equation}
P[X = i] = f(i;n) = \frac{i}{Z}\forall i \in n
\end{equation}
$Z$ is the "partition function" that normalizes the probability mass function.

In [212]:
def pmf_factory(n):
    Z = np.sum(np.arange(1,n+1))
    def _pmf(i):
        if 1 <= i <= n:
            return i/Z
        else:
            return 0
    return _pmf
    
def scaled_pmf_factory(n):
    def _pmf(i):
        if 1 <= i <= n:
            return i
        else:
            return 0
    return  _pmf
    
def generate_transition_matrix(n, low = 100.0, high = 200.0, eps = 1.0e-7):
    q = np.random.uniform(low = low, high = high, size = (n,n))
    for i in range(len(q)):
        q[i] /= np.sum(q[i])
    
    for qi in q:
        _sum = np.sum(qi)
        eps = 1.0e-7
        assert np.abs(_sum - 1.0) < eps, f"Consider providing eps > {eps}"
        
    return q

def expval(n):
    states = np.arange(1, n+1)
    return np.sum(states
                  * np.array(list(map(pmf_factory(n), states))))

assert expval(n) == 67.0

In [131]:
def approx_eq(a, b, eps = 1.0e-7):
    assert eps > 0
    return np.abs(a - b) < eps

In [203]:
class HastingsMetropolisMarkovChain:
    """Assumes orderable states"""
    def __init__(self, q, b, state_space, initial_state = None):
        n = len(state_space)
        state_type = state_space.dtype
        assert n == len(np.unique(state_space)) 
        assert q.shape == (n, n)
        self.b = b
        # naively assumes q is an
        # irreducible Markov Chain transition matrix
        sorted_args = sorted(zip(state_space, q))
        self.state_space = np.array([args[0] for args in sorted_args], dtype = state_type)
        self.q = np.array([args[1] for args in sorted_args], dtype = np.float64)
        if initial_state is None:
            self.state = np.random.choice(state_space)
        else:
            assert initial_state in state_space, f"Invalide initial state: {initial_state}"
            self.state = initial_state
        self.timeseries = [self.state]
        
    def step(self):
        i = np.searchsorted(state_space, self.state)
        next_state = self.__class__.weighted_choice(self.state_space, self.q[i])
        # for readability
        xn = self.state
        x = next_state  
        
        assert self.state_space is not None, "state space is None"
        assert next_state is not None, "next_state is None"
        j = np.searchsorted(self.state_space, next_state)
        
        u = np.random.uniform(low = 0, high = 1, size = 1)
        α = self.b(x) * q[j][i] / (self.b(xn) * q[i][j])
        
        if u < α:
            self.state = next_state
            
        self.timeseries.append(self.state)
        
    @staticmethod
    def weighted_choice(state_space, pmf, eps = 1.0e-7):
        assert approx_eq(np.sum(pmf), 1)
        u = np.random.uniform(low = 0.0, high = 1.0)
        n = len(pmf)
        psum = 0.0
        for i in range(n):
            psum += pmf[i]
            if u < psum:
                return state_space[i]

In [213]:
n = 100
b = scaled_pmf_factory(n)
π = pmf_factory(n)
q = generate_transition_matrix(n)
state_space = np.arange(1,n+1)
assert len(state_space) == n
hmmc_obj = HastingsMetropolisMarkovChain(q, b, state_space, 1)
N = 10**5
for _ in range(N):
    hmmc_obj.step()

In [214]:
print(np.mean(hmmc_obj.timeseries[1000:]))

66.73036329296745
